In [ ]:
from typing import Iterable, Any, Tuple

from rgxlog.primitive_types import DataTypes
from rgxlog.general_utils import QUERY_RESULT_PREFIX
from rgxlog.tests.utils import run_test, run_test_df_compare
import pandas as pd

In [ ]:
def test_issue_80_1() -> None:
    def which_century(year) -> Iterable[int]:
        yield int(year / 100) + 1

    in_out_types = [DataTypes.integer]

    which_century_dict = dict(ie_function=which_century,
                              ie_function_name='which_century',
                              in_rel=in_out_types,
                              out_rel=in_out_types)

    def which_era(cet) -> Iterable[str]:
        if 1 <= cet < 4:
            yield "Targerian Regime"
        elif 4 <= cet < 8:
            yield "Lanister Regime"
        elif 8 <= cet < 12:
            yield "Stark Regime"
        elif 12 <= cet < 16:
            yield "Barathion Regime"
        elif cet >= 16:
            yield "Long Winter"

    which_era_dict = dict(ie_function=which_era,
                          ie_function_name='which_era',
                          in_rel=[DataTypes.integer],
                          out_rel=[DataTypes.string])

    commands = """new event(str, int)
                        event("First Dragon", 250)
                        event("Mad king", 390)
                        event("Winter came", 1750)
                        event("Hodor", 999)
                        event("Joffery died", 799)
                        
                        new important_year(int)
                        important_year(999)
                        important_year(1750)
                        important_year(250)
                        
                        
                        important_events(EVE, Y) <- event(EVE, Y), important_year(Y)
                        
                        important_events_per_cet(EVE, CET) <- important_events(EVE, Y), which_century(Y) -> (CET)
                        ?important_events_per_cet(EVE, CET)
            """
    commands2 = """
                        important_events_per_era(EVE, ERA) <- important_events_per_cet(EVE, CET), which_era(CET) -> (ERA)
                        ?important_events_per_era(EVE, ERA)
            """
    expected_result = f"""{QUERY_RESULT_PREFIX}'important_events_per_cet(EVE, CET)':
                         EVE      |   CET
                    --------------+-------
                     First Dragon |     3
                     Winter came  |    18
                        Hodor     |    10
         """

    expected_result2 = f"""{QUERY_RESULT_PREFIX}'important_events_per_era(EVE, ERA)':
                         EVE      |       ERA
                    --------------+------------------
                     First Dragon | Targerian Regime
                     Winter came  |   Long Winter 
                        Hodor     |   Stark Regime
        """

    session = run_test(commands, expected_result, [which_century_dict])

    run_test(commands2, expected_result2, [which_era_dict], session=session)

test_issue_80_1()

printing results for query 'important_events_per_cet(EVE, CET)':
     EVE      |   CET
--------------+-------
 First Dragon |     3
 Winter came  |    18
    Hodor     |    10

printing results for query 'important_events_per_era(EVE, ERA)':
     EVE      |       ERA
--------------+------------------
 First Dragon | Targerian Regime
 Winter came  |   Long Winter
    Hodor     |   Stark Regime



In [ ]:
def test_complex_history_analysis() -> None:
    def get_population_growth_rate(year) -> Iterable[int]:
        if year <= 1700:
            yield 1
        elif 1700 < year <= 1900:
            yield 3
        elif 1900 < year <= 2000:
            yield 5
        else:  
            yield 1

    population_growth_dict = dict(ie_function=get_population_growth_rate,
                                  ie_function_name='get_population_growth_rate',
                                  in_rel=[DataTypes.integer],
                                  out_rel=[DataTypes.integer])

    def historical_event(year) -> Iterable[str]:
        if year == 1600:
            yield "Discovery of New Land"
        elif year == 1750:
            yield "Industrial Revolution"
        elif year == 1920:
            yield "Roaring Twenties"
        elif year == 2000:
            yield "Y2K Bug Panic"
        else:
            yield "No significant event"

    historical_event_dict = dict(ie_function=historical_event,
                                 ie_function_name='historical_event',
                                 in_rel=[DataTypes.integer],
                                 out_rel=[DataTypes.string])

    commands = """new year_event(int, str)
                        year_event(1600, "Discovery of New Land")
                        year_event(1750, "Industrial Revolution")
                        year_event(1920, "Roaring Twenties")
                        year_event(2000, "Y2K Bug Panic")
                        
                        new population_year(int)
                        population_year(1600)
                        population_year(1700)
                        population_year(1800)
                        population_year(1900)
                        population_year(2000)
                        population_year(1750)
                        population_year(1920)
                        
                        population_growth_rate(Y,GROWTH) <- population_year(Y) ,get_population_growth_rate(Y) -> (GROWTH)
                        
                        historical_event(Y, EVENT) <- year_event(Y, EVENT)
                        
                        important_years(Y, EVENT, GROWTH) <- population_growth_rate(Y, GROWTH), historical_event(Y, EVENT)
                        
                        ?important_years(Y, EVENT, GROWTH)
            """
    expected_result = f"""{QUERY_RESULT_PREFIX}'important_years(Y, EVENT, GROWTH)':
                          Y |         EVENT         |   GROWTH
                      ------+-----------------------+----------
                       1600 | Discovery of New Land |        1
                       1750 | Industrial Revolution |        3
                       1920 |   Roaring Twenties    |        5
                       2000 |     Y2K Bug Panic     |        5
         """

    session = run_test(commands, expected_result, [population_growth_dict, historical_event_dict])

test_complex_history_analysis()

printing results for query 'important_years(Y, EVENT, GROWTH)':
    Y |         EVENT         |   GROWTH
------+-----------------------+----------
 1600 | Discovery of New Land |        1
 1750 | Industrial Revolution |        3
 1920 |   Roaring Twenties    |        5
 2000 |     Y2K Bug Panic     |        5



In [ ]:
def test_advanced_history_analysis() -> None:
    def get_population_growth_rate(year) -> Iterable[int]:
        if year <= 1700:
            yield 1
        elif 1700 < year <= 1900:
            yield 3
        elif 1900 < year <= 2000:
            yield 5
        else:
            yield 1

    population_growth_dict = dict(ie_function=get_population_growth_rate,
                                  ie_function_name='get_population_growth_rate',
                                  in_rel=[DataTypes.integer],
                                  out_rel=[DataTypes.integer])

    def technological_advancement(year) -> Iterable[str]:
        if year <= 1700:
            yield "Low"
        elif 1700 < year <= 1900:
            yield "Moderate"
        elif 1900 < year <= 2000:
            yield "High"
        else:
            yield "Low"

    technological_advancement_dict = dict(ie_function=technological_advancement,
                                         ie_function_name='technological_advancement',
                                         in_rel=[DataTypes.integer],
                                         out_rel=[DataTypes.string])

    commands = """new year_event(int, str)
                        year_event(1600, "Discovery of New Land")
                        year_event(1750, "Industrial Revolution")
                        year_event(1920, "Roaring Twenties")
                        year_event(2000, "Y2K Bug Panic")
                        
                        new population_year(int)
                        population_year(1600)
                        population_year(1700)
                        population_year(1800)
                        population_year(1900)
                        population_year(2000)
                        population_year(1750)
                        population_year(1920)
                        
                        population_growth_rate(Y, GROWTH) <- population_year(Y), get_population_growth_rate(Y) -> (GROWTH)
                        
                        technological_advancement(Y, TECH) <- year_event(Y, Z), technological_advancement(Y) -> (TECH)
                        
                        important_years(Y, EVENT, GROWTH, TECH) <- population_growth_rate(Y, GROWTH), year_event(Y, EVENT), technological_advancement(Y, TECH)
                        
                        ?important_years(Y, EVENT, GROWTH, TECH)
            """
    expected_result = f"""{QUERY_RESULT_PREFIX}'important_years(Y, EVENT, GROWTH, TECH)':
                          Y |         EVENT         |   GROWTH |   TECH
                      ------+-----------------------+----------+----------
                       1600 | Discovery of New Land |        1 |   Low
                       1750 | Industrial Revolution |        3 | Moderate
                       1920 |   Roaring Twenties    |        5 |   High
                       2000 |     Y2K Bug Panic     |        5 |   High
         """

    session = run_test(commands, expected_result, [population_growth_dict, technological_advancement_dict])

test_advanced_history_analysis()

printing results for query 'important_years(Y, EVENT, GROWTH, TECH)':
    Y |         EVENT         |   GROWTH |   TECH
------+-----------------------+----------+----------
 1600 | Discovery of New Land |        1 |   Low
 1750 | Industrial Revolution |        3 | Moderate
 1920 |   Roaring Twenties    |        5 |   High
 2000 |     Y2K Bug Panic     |        5 |   High



In [ ]:
def test_football_analysis() -> None:
    def get_match_result(home_team, away_team, home_goals, away_goals) -> Iterable[str]:
        if home_goals > away_goals:
            yield "Home Win"
        elif home_goals < away_goals:
            yield "Away Win"
        else:
            yield "Draw"

    match_result_dict = dict(ie_function=get_match_result,
                             ie_function_name='get_match_result',
                             in_rel=[DataTypes.string, DataTypes.string, DataTypes.integer, DataTypes.integer],
                             out_rel=[DataTypes.string])

    def get_player_position(player_name) -> Iterable[str]:
        if player_name in ["Lionel Messi", "Cristiano Ronaldo"]:
            yield "Forward"
        elif player_name in ["Virgil van Dijk", "Sergio Ramos"]:
            yield "Defender"
        else:
            yield "Unknown"

    player_position_dict = dict(ie_function=get_player_position,
                                ie_function_name='get_player_position',
                                in_rel=[DataTypes.string],
                                out_rel=[DataTypes.string])

    commands = """new team(str)
                        team("Barcelona")
                        team("Real Madrid")
                        team("Liverpool")
                        team("Juventus")
                        
                        new player(str, str)
                        player("Lionel Messi", "Barcelona")
                        player("Cristiano Ronaldo", "Juventus")
                        player("Virgil van Dijk", "Liverpool")
                        player("Sergio Ramos", "Real Madrid")
                        
                        new match(str, str, int, int)
                        match("Barcelona", "Real Madrid", 3, 2)
                        match("Real Madrid", "Barcelona", 1, 1)
                        match("Liverpool", "Juventus", 2, 0)
                        match("Juventus", "Liverpool", 0, 1)
                        
                        match_result(HT, AT, HG, AG, RESULT) <- match(HT, AT, HG, AG), get_match_result(HT, AT, HG, AG) -> (RESULT)
                        
                        player_position(P, POS) <- player(P, X), get_player_position(P) -> (POS)
                        
                        ?match_result(HT, AT, HG, AG, RESULT)
                        ?player_position(P, POS)
            """
    expected_result = f"""{QUERY_RESULT_PREFIX}'match_result(HT, AT, HG, AG, RESULT)':
                         HT      |     AT      |   HG |   AG |  RESULT
                    -------------+-------------+------+------+----------
                      Barcelona  | Real Madrid |    3 |    2 | Home Win
                     Real Madrid |  Barcelona  |    1 |    1 |   Draw
                      Liverpool  |  Juventus   |    2 |    0 | Home Win
                      Juventus   |  Liverpool  |    0 |    1 | Away Win

                    {QUERY_RESULT_PREFIX}'player_position(P, POS)':
                             P         |   POS
                    -------------------+----------
                       Lionel Messi    | Forward
                     Cristiano Ronaldo | Forward
                      Virgil van Dijk  | Defender
                       Sergio Ramos    | Defender
         """

    session = run_test(commands, expected_result, [match_result_dict, player_position_dict])

test_football_analysis()

printing results for query 'match_result(HT, AT, HG, AG, RESULT)':
     HT      |     AT      |   HG |   AG |  RESULT
-------------+-------------+------+------+----------
  Barcelona  | Real Madrid |    3 |    2 | Home Win
 Real Madrid |  Barcelona  |    1 |    1 |   Draw
  Liverpool  |  Juventus   |    2 |    0 | Home Win
  Juventus   |  Liverpool  |    0 |    1 | Away Win

printing results for query 'player_position(P, POS)':
         P         |   POS
-------------------+----------
   Lionel Messi    | Forward
 Cristiano Ronaldo | Forward
  Virgil van Dijk  | Defender
   Sergio Ramos    | Defender



In [ ]:
def test_movie_analysis() -> None:
    def get_movie_genre(movie_title) -> Iterable[str]:
        if movie_title == "The Shawshank Redemption":
            yield "Drama"
        elif movie_title == "Inception":
            yield "Sci-Fi"
        elif movie_title == "The Dark Knight":
            yield "Action"
        else:
            yield "Unknown"

    movie_genre_dict = dict(ie_function=get_movie_genre,
                            ie_function_name='get_movie_genre',
                            in_rel=[DataTypes.string],
                            out_rel=[DataTypes.string])

    def get_actor_role(actor_name) -> Iterable[str]:
        if actor_name == "Tom Hanks":
            yield "Protagonist"
        elif actor_name == "Leonardo DiCaprio":
            yield "Lead"
        else:
            yield "Supporting"

    actor_role_dict = dict(ie_function=get_actor_role,
                          ie_function_name='get_actor_role',
                          in_rel=[DataTypes.string],
                          out_rel=[DataTypes.string])

    commands = """new movie(str)
                        movie("The Shawshank Redemption")
                        movie("Inception")
                        movie("The Dark Knight")
                        movie("Interstellar")
                        
                        new actor(str)
                        actor("Tom Hanks")
                        actor("Leonardo DiCaprio")
                        actor("Christian Bale")
                        
                        new cast(str, str)
                        cast("The Shawshank Redemption", "Tom Hanks")
                        cast("Inception", "Leonardo DiCaprio")
                        cast("The Dark Knight", "Christian Bale")
                        cast("Interstellar", "Tom Hanks")
                        
                        movie_genre(M, GENRE) <- movie(M), get_movie_genre(M) -> (GENRE)
                        
                        actor_role(A, ROLE) <- actor(A), get_actor_role(A) -> (ROLE)
                        
                        ?movie_genre(M, GENRE)
                        ?actor_role(A, ROLE)
            """
    expected_result = f"""{QUERY_RESULT_PREFIX}'movie_genre(M, GENRE)':
                                M             |  GENRE
                    --------------------------+---------
                     The Shawshank Redemption |  Drama
                            Inception         | Sci-Fi
                         The Dark Knight      | Action
                           Interstellar       | Unknown

                    {QUERY_RESULT_PREFIX}'actor_role(A, ROLE)':
                             A         |    ROLE
                    -------------------+-------------
                         Tom Hanks     | Protagonist
                     Leonardo DiCaprio |    Lead
                      Christian Bale   | Supporting
         """

    session = run_test(commands, expected_result, [movie_genre_dict, actor_role_dict])

test_movie_analysis()

printing results for query 'movie_genre(M, GENRE)':
            M             |  GENRE
--------------------------+---------
 The Shawshank Redemption |  Drama
        Inception         | Sci-Fi
     The Dark Knight      | Action
       Interstellar       | Unknown

printing results for query 'actor_role(A, ROLE)':
         A         |    ROLE
-------------------+-------------
     Tom Hanks     | Protagonist
 Leonardo DiCaprio |    Lead
  Christian Bale   | Supporting



In [ ]:
def test_game_of_thrones_analysis() -> None:
    def get_house_allegiance(character) -> Iterable[str]:
        allegiances = {
            "Jon Snow": "Stark",
            "Daenerys Targaryen": "Targaryen",
            "Cersei Lannister": "Lannister",
            "Tyrion Lannister": "Lannister",
            "Arya Stark": "Stark",
            "Sansa Stark": "Stark",
            "Jaime Lannister": "Lannister"
        }
        if character in allegiances:
            yield allegiances[character]
        else:
            yield "Unknown"

    house_allegiance_dict = dict(ie_function=get_house_allegiance,
                                  ie_function_name='get_house_allegiance',
                                  in_rel=[DataTypes.string],
                                  out_rel=[DataTypes.string])

    def get_kingdom(house) -> Iterable[str]:
        kingdoms = {
            "Stark": "North",
            "Targaryen": "Westeros",
            "Lannister": "Westeros"
        }
        if house in kingdoms:
            yield kingdoms[house]
        else:
            yield "Unknown"

    house_kingdom_dict = dict(ie_function=get_kingdom,
                              ie_function_name='get_kingdom',
                              in_rel=[DataTypes.string],
                              out_rel=[DataTypes.string])

    commands = """new character(str)
                        character("Jon Snow")
                        character("Daenerys Targaryen")
                        character("Cersei Lannister")
                        character("Tyrion Lannister")
                        character("Arya Stark")
                        character("Sansa Stark")
                        character("Jaime Lannister")
                        character("Eddard Stark")
                        
                        house_allegiance(NAME, HOUSE) <- character(NAME), get_house_allegiance(NAME) -> (HOUSE)
                        house_kingdom(HOUSE, KINGDOM) <- house_allegiance(NAME,HOUSE) ,get_kingdom(HOUSE) -> (KINGDOM)
                        
                        ?house_allegiance(NAME, HOUSE)
                        ?house_kingdom(HOUSE, KINGDOM)
            """
    expected_result = f"""{QUERY_RESULT_PREFIX}'house_allegiance(NAME, HOUSE)':
                        NAME        |   HOUSE
                --------------------+-----------
                      Jon Snow      |   Stark
                 Daenerys Targaryen | Targaryen
                  Cersei Lannister  | Lannister
                  Tyrion Lannister  | Lannister
                     Arya Stark     |   Stark
                    Sansa Stark     |   Stark
                  Jaime Lannister   | Lannister
                    Eddard Stark    |  Unknown

                    {QUERY_RESULT_PREFIX}'house_kingdom(HOUSE, KINGDOM)':
                       HOUSE   |  KINGDOM
                    -----------+-----------
                       Stark   |   North
                     Targaryen | Westeros
                     Lannister | Westeros
                      Unknown  |  Unknown
         """

    session = run_test(commands, expected_result, [house_allegiance_dict, house_kingdom_dict])

test_game_of_thrones_analysis()

printing results for query 'house_allegiance(NAME, HOUSE)':
        NAME        |   HOUSE
--------------------+-----------
      Jon Snow      |   Stark
 Daenerys Targaryen | Targaryen
  Cersei Lannister  | Lannister
  Tyrion Lannister  | Lannister
     Arya Stark     |   Stark
    Sansa Stark     |   Stark
  Jaime Lannister   | Lannister
    Eddard Stark    |  Unknown

printing results for query 'house_kingdom(HOUSE, KINGDOM)':
   HOUSE   |  KINGDOM
-----------+-----------
   Stark   |   North
 Targaryen | Westeros
 Lannister | Westeros
  Unknown  |  Unknown



In [ ]:
def test_dinosaur_analysis() -> None:
    def carnivore_diet(dinosaur) -> Iterable[int]:
        carnivores = ["Tyrannosaurus", "Velociraptor", "Spinosaurus"]
        if dinosaur in carnivores:
            yield 1
        else:
            yield 0

    carnivore_diet_dict = dict(ie_function=carnivore_diet,
                               ie_function_name='carnivore_diet',
                               in_rel=[DataTypes.string],
                               out_rel=[DataTypes.integer])

    def herbivore_diet(dinosaur) -> Iterable[int]:
        herbivores = ["Triceratops", "Brachiosaurus", "Stegosaurus"]
        if dinosaur in herbivores:
            yield 1
        else:
            yield 0

    herbivore_diet_dict = dict(ie_function=herbivore_diet,
                              ie_function_name='herbivore_diet',
                              in_rel=[DataTypes.string],
                              out_rel=[DataTypes.integer])

    def get_dinosaur_period(dinosaur) -> Iterable[str]:
        periods = {
            "Tyrannosaurus": "Late Cretaceous",
            "Triceratops": "Late Cretaceous",
            "Velociraptor": "Late Cretaceous",
            "Brachiosaurus": "Jurassic",
            "Stegosaurus": "Jurassic",
            "Spinosaurus": "Cretaceous"
        }
        if dinosaur in periods:
            yield periods[dinosaur]
        else:
            yield "Unknown"

    dinosaur_period_dict = dict(ie_function=get_dinosaur_period,
                               ie_function_name='get_dinosaur_period',
                               in_rel=[DataTypes.string],
                               out_rel=[DataTypes.string])

    def get_dinosaur_classification(dinosaur) -> Iterable[str]:
        classifications = {
            "Tyrannosaurus": "Theropod",
            "Triceratops": "Ceratopsian",
            "Velociraptor": "Theropod",
            "Brachiosaurus": "Sauropod",
            "Stegosaurus": "Stegosaurid",
            "Spinosaurus": "Spinosaurid"
        }
        if dinosaur in classifications:
            yield classifications[dinosaur]
        else:
            yield "Unknown"

    dinosaur_classification_dict = dict(ie_function=get_dinosaur_classification,
                                       ie_function_name='get_dinosaur_classification',
                                       in_rel=[DataTypes.string],
                                       out_rel=[DataTypes.string])

    commands = """new dinosaur(str)
                        dinosaur("Tyrannosaurus")
                        dinosaur("Triceratops")
                        dinosaur("Velociraptor")
                        dinosaur("Brachiosaurus")
                        dinosaur("Stegosaurus")
                        dinosaur("Spinosaurus")
                        
                        carnivore_diet(DINO, CARNIVORE) <- dinosaur(DINO), carnivore_diet(DINO) -> (CARNIVORE)
                        herbivore_diet(DINO, HERBIVORE) <- dinosaur(DINO), herbivore_diet(DINO) -> (HERBIVORE)
                        dinosaur_period(DINO, PERIOD) <- dinosaur(DINO), get_dinosaur_period(DINO) -> (PERIOD)
                        dinosaur_classification(DINO, CLASS) <- dinosaur(DINO), get_dinosaur_classification(DINO) -> (CLASS)
                        
                        ?carnivore_diet(DINO, CARNIVORE)
                        ?herbivore_diet(DINO, HERBIVORE)
                        ?dinosaur_period(DINO, PERIOD)
                        ?dinosaur_classification(DINO, CLASS)
            """
    expected_result = f"""{QUERY_RESULT_PREFIX}'carnivore_diet(DINO, CARNIVORE)':
                         DINO      |   CARNIVORE
                    ---------------+-------------
                     Tyrannosaurus |           1
                      Triceratops  |           0
                     Velociraptor  |           1
                     Brachiosaurus |           0
                      Stegosaurus  |           0
                      Spinosaurus  |           1

                    {QUERY_RESULT_PREFIX}'herbivore_diet(DINO, HERBIVORE)':
                         DINO      |   HERBIVORE
                    ---------------+-------------
                     Tyrannosaurus |           0
                      Triceratops  |           1
                     Velociraptor  |           0
                     Brachiosaurus |           1
                      Stegosaurus  |           1
                      Spinosaurus  |           0

                    {QUERY_RESULT_PREFIX}'dinosaur_period(DINO, PERIOD)':
                         DINO      |     PERIOD
                    ---------------+-----------------
                     Tyrannosaurus | Late Cretaceous
                      Triceratops  | Late Cretaceous
                     Velociraptor  | Late Cretaceous
                     Brachiosaurus |    Jurassic
                      Stegosaurus  |    Jurassic
                      Spinosaurus  |   Cretaceous

                    {QUERY_RESULT_PREFIX}'dinosaur_classification(DINO, CLASS)':
                         DINO      |    CLASS
                    ---------------+-------------
                     Tyrannosaurus |  Theropod
                      Triceratops  | Ceratopsian
                     Velociraptor  |  Theropod
                     Brachiosaurus |  Sauropod
                      Stegosaurus  | Stegosaurid
                      Spinosaurus  | Spinosaurid
         """

    session = run_test(commands, expected_result, [carnivore_diet_dict, herbivore_diet_dict,
                                                   dinosaur_period_dict, dinosaur_classification_dict])
    
test_dinosaur_analysis()

printing results for query 'carnivore_diet(DINO, CARNIVORE)':
     DINO      |   CARNIVORE
---------------+-------------
 Tyrannosaurus |           1
  Triceratops  |           0
 Velociraptor  |           1
 Brachiosaurus |           0
  Stegosaurus  |           0
  Spinosaurus  |           1

printing results for query 'herbivore_diet(DINO, HERBIVORE)':
     DINO      |   HERBIVORE
---------------+-------------
 Tyrannosaurus |           0
  Triceratops  |           1
 Velociraptor  |           0
 Brachiosaurus |           1
  Stegosaurus  |           1
  Spinosaurus  |           0

printing results for query 'dinosaur_period(DINO, PERIOD)':
     DINO      |     PERIOD
---------------+-----------------
 Tyrannosaurus | Late Cretaceous
  Triceratops  | Late Cretaceous
 Velociraptor  | Late Cretaceous
 Brachiosaurus |    Jurassic
  Stegosaurus  |    Jurassic
  Spinosaurus  |   Cretaceous

printing results for query 'dinosaur_classification(DINO, CLASS)':
     DINO      |    CLASS
-----

In [ ]:
def test_complex_economic_analysis() -> None:
    def calc_economic_index(interest, inflation) -> Iterable[int]:
        yield interest + inflation

    calc_economic_index_dict = dict(ie_function=calc_economic_index,
                                    ie_function_name='calc_economic_index',
                                    in_rel=[DataTypes.integer, DataTypes.integer],
                                    out_rel=[DataTypes.integer])

    def classify_economic_index(index) -> Iterable[str]:
        if index > 15:
            yield "Bad"
        else:
            yield "Good"

    classify_economic_index_dict = dict(ie_function=classify_economic_index,
                                        ie_function_name='classify_economic_index',
                                        in_rel=[DataTypes.integer],
                                        out_rel=[DataTypes.string])

    commands = """new economy_data(int, int, int)  # year, interest, inflation
                        economy_data(2000, 4, 3)
                        economy_data(2008, 8, 10)
                        economy_data(2019, 3, 2)

                        complex_economic_status(YEAR, STATUS) <- economy_data(YEAR, INTEREST, INFLATION), calc_economic_index(INTEREST, INFLATION) -> (INDEX), classify_economic_index(INDEX) -> (STATUS)
                        
                        ?complex_economic_status(YEAR, STATUS)
              """
    expected_result = f"""{QUERY_RESULT_PREFIX}'complex_economic_status(YEAR, STATUS)':
                                   YEAR |  STATUS
                                --------+----------
                                2000 |   Good
                                2008 |   Bad
                                2019 |   Good
                  """

    session = run_test(commands, expected_result, [calc_economic_index_dict, classify_economic_index_dict])

test_complex_economic_analysis()

printing results for query 'complex_economic_status(YEAR, STATUS)':
   YEAR |  STATUS
--------+----------
   2000 |   Good
   2008 |   Bad
   2019 |   Good



In [ ]:
def test_complex_historical_and_technological_analysis() -> None:
    def combine_impact(year, growth) -> Iterable[int]:
        yield year + growth

    combine_impact_dict = dict(ie_function=combine_impact,
                               ie_function_name='combine_impact',
                               in_rel=[DataTypes.integer, DataTypes.integer],
                               out_rel=[DataTypes.integer])

    def classify_impact(impact) -> Iterable[str]:
        if impact > 2200:
            yield "Significant"
        else:
            yield "Moderate"

    classify_impact_dict = dict(ie_function=classify_impact,
                                ie_function_name='classify_impact',
                                in_rel=[DataTypes.integer],
                                out_rel=[DataTypes.string])

    commands = """new year_event(int, str)  
                        year_event(1600, "Discovery of New Land")
                        year_event(1750, "Industrial Revolution")

                        new population_year(int, int)
                        population_year(1600, 1)
                        population_year(1750, 3)

                        complex_impact_years(YEAR, EVENT, IMPACT_TYPE) <- year_event(YEAR, EVENT), population_year(YEAR, GROWTH), combine_impact(YEAR, GROWTH) -> (IMPACT), classify_impact(IMPACT) -> (IMPACT_TYPE)
                        
                        ?complex_impact_years(YEAR, EVENT, IMPACT_TYPE)
              """
    expected_result = f"""{QUERY_RESULT_PREFIX}'complex_impact_years(YEAR, EVENT, IMPACT_TYPE)':
                               YEAR |         EVENT         |  IMPACT_TYPE
                            --------+-----------------------+---------------
                            1600 | Discovery of New Land |   Moderate
                            1750 | Industrial Revolution |   Moderate
                  """

    session = run_test(commands, expected_result, [combine_impact_dict, classify_impact_dict])

test_complex_historical_and_technological_analysis()

printing results for query 'complex_impact_years(YEAR, EVENT, IMPACT_TYPE)':
   YEAR |         EVENT         |  IMPACT_TYPE
--------+-----------------------+---------------
   1600 | Discovery of New Land |   Moderate
   1750 | Industrial Revolution |   Moderate



In [ ]:
def test_complex_space_exploration_analysis() -> None:
    def calc_travel_time(distance) -> Iterable[int]:
        yield int(distance / 1000)

    calc_travel_time_dict = dict(ie_function=calc_travel_time,
                                 ie_function_name='calc_travel_time',
                                 in_rel=[DataTypes.integer],
                                 out_rel=[DataTypes.integer])

    def analyze_elements(elements) -> Iterable[int]:
        score = 0
        if 'Carbon' in elements:
            score += 5
        if 'Water' in elements:
            score += 5
        yield score

    analyze_elements_dict = dict(ie_function=analyze_elements,
                                 ie_function_name='analyze_elements',
                                 in_rel=[DataTypes.string],
                                 out_rel=[DataTypes.integer])

    def calculate_life_probability(age, element_score, travel_time) -> Iterable[int]:
        yield int((element_score * 2) - (age / 1000) - travel_time)

    calculate_life_probability_dict = dict(ie_function=calculate_life_probability,
                                           ie_function_name='calculate_life_probability',
                                           in_rel=[DataTypes.integer, DataTypes.integer, DataTypes.integer],
                                           out_rel=[DataTypes.integer])

    commands = """new exoplanet(str, int, int, str)  # name, distance, age, elements
                        exoplanet("PlanetA", 4000, 10000, "Carbon,Water")
                        exoplanet("PlanetB", 9000, 5000, "Hydrogen,Silicon")
                        
                        travel_time(NAME, TIME) <- exoplanet(NAME, DIST, AGE, ELEM), calc_travel_time(DIST) -> (TIME)
                        element_score(NAME, SCORE) <- exoplanet(NAME, DIST, AGE, ELEM), analyze_elements(ELEM) -> (SCORE)
                        life_probability(NAME, PROB) <- exoplanet(NAME, DIST, AGE, ELEM), travel_time(NAME, TIME), element_score(NAME, SCORE), calculate_life_probability(AGE, SCORE, TIME) -> (PROB)
                        ?life_probability(NAME, PROB)
              """
    expected_result = f"""{QUERY_RESULT_PREFIX}'life_probability(NAME, PROB)':
                            NAME   |   PROB
                            ---------+--------
                            PlanetA |      6
                            PlanetB |    -14
                  """

    session = run_test(commands, expected_result, [calc_travel_time_dict, analyze_elements_dict, calculate_life_probability_dict])

test_complex_space_exploration_analysis()

printing results for query 'life_probability(NAME, PROB)':
  NAME   |   PROB
---------+--------
 PlanetA |      6
 PlanetB |    -14



In [ ]:
def test_complex_analysis() -> None:
    def price_category(price) -> Iterable[int]:
        if price < 100:
            yield 1
        elif price < 200:
            yield 2
        else:
            yield 3

    price_category_dict = dict(ie_function=price_category,
                               ie_function_name='price_category',
                               in_rel=[DataTypes.integer],
                               out_rel=[DataTypes.integer])

    def rating_category(rating) -> Iterable[str]:
        if rating < 3:
            yield "Low"
        elif rating < 4:
            yield "Medium"
        else:
            yield "High"

    rating_category_dict = dict(ie_function=rating_category,
                                ie_function_name='rating_category',
                                in_rel=[DataTypes.integer],
                                out_rel=[DataTypes.string])

    def calculate_discount(price_category, rating_category) -> Iterable[int]:
        if price_category == 1 and rating_category == "High":
            yield 20
        elif price_category == 2 and rating_category == "Medium":
            yield 10
        elif price_category == 3 and rating_category == "Low":
            yield 5
        else:
            yield 0

    calculate_discount_dict = dict(ie_function=calculate_discount,
                                   ie_function_name='calculate_discount',
                                   in_rel=[DataTypes.integer, DataTypes.string],
                                   out_rel=[DataTypes.integer])

    commands = """
               new product(str, int, int)
               product("Laptop", 999, 5)
               product("Phone", 499, 4)
               product("Watch", 199, 3)
               product("Earphones", 49, 2)
               
               price_category(P, CAT) <- product(P, PRICE, R), price_category(PRICE) -> (CAT)
               
               rating_category(P, CAT) <- product(P, PRICE, R), rating_category(R) -> (CAT)
               
               product_discount(P, CAT, DISCOUNT) <- price_category(P, CAT), rating_category(P, RCAT), calculate_discount(CAT, RCAT) -> (DISCOUNT)
               
               ?product_discount(P, CAT, DISCOUNT)
               """

    expected_result = {
        'P': ['Laptop', 'Phone', 'Watch', 'Earphones'],
        'CAT': [3, 3, 2, 1],
        'DISCOUNT': [0, 0, 10, 0]
    }

    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, [price_category_dict, rating_category_dict, calculate_discount_dict])

test_complex_analysis()

In [ ]:
def test_complicated_analysis1() -> None:
    def ie_func1(age) -> Iterable[int]:
        yield age * 2
    
    def ie_func2(age, doubled_age) -> Iterable[str]:
        if doubled_age > 50:
            yield "Old"
        else:
            yield "Young"
    ie_func1_dict = dict(ie_function=ie_func1,
                     ie_function_name='ie_func1',
                     in_rel=[DataTypes.integer],
                     out_rel=[DataTypes.integer])

    ie_func2_dict = dict(ie_function=ie_func2,
                        ie_function_name='ie_func2',
                        in_rel=[DataTypes.integer, DataTypes.integer],
                        out_rel=[DataTypes.string])

        
    commands = """
               new person(str, int)
               new job(str, int)
               new hobby(str, str)
               
               person("Alice", 30)
               person("Bob", 25)
               person("Carol", 55)
               
               job("Alice", 5000)
               job("Bob", 6000)
               job("Carol", 10000)
               
               hobby("Alice", "Reading")
               hobby("Carol", "Swimming")
               
               sample_rule1(P, S, H) <- job(P, S), person(P, A), ie_func1(A) -> (AD), hobby(P, H), ie_func2(A, AD) -> (AGE_GROUP)
               
               ?sample_rule1(P, S, H)
               """
    
    expected_result = {
        'P': ['Alice', 'Carol'],
        'S': [5000, 10000],
        'H': ['Reading', 'Swimming']
    }
    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, [ie_func1_dict, ie_func2_dict])

test_complicated_analysis1()

In [ ]:
def test_complicated_analysis2() -> None:
    def ie_func3(salary) -> Iterable[int]:
        yield salary - 1000
    
    def ie_func4(age, adjusted_salary) -> Iterable[str]:
        if age > 40:
            yield "Senior"
        else:
            yield "Junior"
        
    ie_func3_dict = dict(ie_function=ie_func3,
                     ie_function_name='ie_func3',
                     in_rel=[DataTypes.integer],
                     out_rel=[DataTypes.integer])

    ie_func4_dict = dict(ie_function=ie_func4,
                        ie_function_name='ie_func4',
                        in_rel=[DataTypes.integer, DataTypes.integer],
                        out_rel=[DataTypes.string])

    commands = """
               new job(str, int)
               job("Alice", 5000)
               job("Bob", 6000)
               job("Carol", 10000)

               new person(str, int)
               person("Alice", 35)
               person("Bob", 30)
               person("Carol", 45)
               
               sample_rule2(P, S, G) <- job(P, S), person(P, A), ie_func3(S) -> (AS), ie_func4(A, AS) -> (G)
               
               ?sample_rule2(P, S, G)
               """
    
    expected_result = {
        'P': ['Alice', 'Bob', 'Carol'],
        'S': [5000, 6000, 10000],
        'G': ['Junior', 'Junior', 'Senior']
    }
    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, [ie_func3_dict, ie_func4_dict])

test_complicated_analysis2()

In [ ]:
def test_complicated_analysis3() -> None:
    def ie_func1(age) -> Iterable[int]:
        yield age + 1

    def ie_func2(course, increased_age) -> Iterable[str]:
        if increased_age > 26:
            yield f"Senior-{course}"
        else:
            yield f"Junior-{course}"

    ie_func1_dict = dict(ie_function=ie_func1,
                     ie_function_name='ie_func1',
                     in_rel=[DataTypes.integer],
                     out_rel=[DataTypes.integer])

    ie_func2_dict = dict(ie_function=ie_func2,
                        ie_function_name='ie_func2',
                        in_rel=[DataTypes.string, DataTypes.integer],
                        out_rel=[DataTypes.string])

    commands = """
               new student(str, int)
               student("Alice", 25)
               student("Bob", 22)
               student("Charlie", 28)
               
               new course(str, str)
               course("CS101", "Computer Science")
               course("EN101", "English")
               course("MA101", "Mathematics")
               
               new enrolled(str, str)
               enrolled("Alice", "CS101")
               enrolled("Alice", "EN101")
               enrolled("Bob", "MA101")
               enrolled("Charlie", "EN101")
               
               sample_rule3(S, A, G) <- student(S, A), enrolled(S, C), course(C, T), ie_func1(A) -> (IA), ie_func2(T, IA) -> (G)
                                        
               ?sample_rule3(S, A, G)
               """

    expected_result = {
        'S': ['Alice', 'Alice', 'Bob', 'Charlie'],
        'A': [25, 25, 22, 28],
        'G': ['Junior-Computer Science', 'Junior-English', 'Junior-Mathematics', 'Senior-English']
    }

    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, [ie_func1_dict, ie_func2_dict])

test_complicated_analysis3()

In [ ]:
def test_ie_func_complex_analysis1() -> None:
    from typing import Iterable
    import pandas as pd
    
    def ie_func1(x) -> Iterable[int]:
        yield x * 2

    def ie_func2(y, z) -> Iterable[int]:
        yield y + z

    ie_func1_dict = dict(ie_function=ie_func1,
                         ie_function_name='ie_func1',
                         in_rel=[DataTypes.integer],
                         out_rel=[DataTypes.integer])

    ie_func2_dict = dict(ie_function=ie_func2,
                         ie_function_name='ie_func2',
                         in_rel=[DataTypes.integer, DataTypes.integer],
                         out_rel=[DataTypes.integer])

    commands = """
               new rel1(int)
               new rel2(int)
               new rel3(int)
               rel1(1)
               rel2(1)
               rel3(2)

               sample_rule1(X, Y, Z) <- rel1(Y), rel2(Z), ie_func1(Z) -> (Y1), rel3(Y1), ie_func2(Y, Y1) -> (X)

               ?sample_rule1(X, Y, Z)
               """

    expected_result = {
        'X': [3],
        'Y': [1],
        'Z': [1]
    }

    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, [ie_func1_dict, ie_func2_dict])

test_ie_func_complex_analysis1()

In [ ]:
def test_extreme_analysis1() -> None:
    def ie_func1(x) -> Iterable[int]:
        yield x * 2

    def ie_func2(x, y) -> Iterable[int]:
        yield x + y

    def ie_func3(x) -> Iterable[int]:
        yield x - 1

    def ie_func4(x, y) -> Iterable[int]:
        yield x * y

    ie_func1_dict = dict(ie_function=ie_func1,
                         ie_function_name='ie_func1',
                         in_rel=[DataTypes.integer],
                         out_rel=[DataTypes.integer])

    ie_func2_dict = dict(ie_function=ie_func2,
                         ie_function_name='ie_func2',
                         in_rel=[DataTypes.integer, DataTypes.integer],
                         out_rel=[DataTypes.integer])

    ie_func3_dict = dict(ie_function=ie_func3,
                         ie_function_name='ie_func3',
                         in_rel=[DataTypes.integer],
                         out_rel=[DataTypes.integer])

    ie_func4_dict = dict(ie_function=ie_func4,
                         ie_function_name='ie_func4',
                         in_rel=[DataTypes.integer, DataTypes.integer],
                         out_rel=[DataTypes.integer])

    commands = """
               new rel1(int)
               new rel2(int)
               new rel3(int)
               new rel4(int)
               rel1(1)
               rel1(2)
               rel2(1)
               rel2(2)
               rel3(2)
               rel4(3)

               sample_rule1(X, Y1, Y2, Z1, Z2, W) <- rel1(Y1), rel2(Z1), ie_func1(Z1) -> (Y2), rel3(Y2), ie_func2(Y1, Y2) -> (Z2),rel4(Z2),ie_func3(Z2) -> (W),ie_func4(Y1, W) -> (X)

               ?sample_rule1(X, Y1, Y2, Z1, Z2, W)
               """

    expected_result = {
        'X': [2],  
        'Y1': [1],
        'Y2': [2],
        'Z1': [1],
        'Z2': [3],
        'W': [2]
    }

    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, 
                                  [ie_func1_dict, ie_func2_dict, ie_func3_dict, ie_func4_dict])
    
test_extreme_analysis1()

In [ ]:
def test_school_complexity1() -> None:
    from typing import Iterable
    import pandas as pd
    
    def calculate_average(math, science) -> Iterable[int]:
        yield (math + science) // 2
    
    def grade_category(avg) -> Iterable[str]:
        if avg >= 90:
            yield "A"
        elif avg >= 75:
            yield "B"
        else:
            yield "C"
            
    def status(math, science, grade) -> Iterable[str]:
        if math >= 90 and science >= 90 and grade == "A":
            yield "Excellent"
        else:
            yield "Good"
    
    calculate_average_dict = dict(ie_function=calculate_average, ie_function_name='calculate_average', in_rel=[DataTypes.integer, DataTypes.integer], out_rel=[DataTypes.integer])
    grade_category_dict = dict(ie_function=grade_category, ie_function_name='grade_category', in_rel=[DataTypes.integer], out_rel=[DataTypes.string])
    status_dict = dict(ie_function=status, ie_function_name='status', in_rel=[DataTypes.integer, DataTypes.integer, DataTypes.string], out_rel=[DataTypes.string])
    
    commands = """
               new student(str, int, int)
               student("Alice", 85, 92)
               student("Bob", 76, 88)
               student("Carol", 95, 91)
               
               average_grade(S, M, SC, AVG) <- student(S, M, SC), calculate_average(M, SC) -> (AVG)
               
               grade_status(S, AVG, GRADE, STATUS) <- average_grade(S, M, SC, AVG), grade_category(AVG) -> (GRADE), status(M, SC, GRADE) -> (STATUS)
               
               ?grade_status(S, AVG, GRADE, STATUS)
               """
    expected_result = {
        'S': ['Alice', 'Bob', 'Carol'],
        'AVG': [88, 82, 93],
        'GRADE': ['B', 'B', 'A'],
        'STATUS': ['Good', 'Good', 'Excellent']
    }
    
    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, [calculate_average_dict, grade_category_dict, status_dict])

test_school_complexity1()

In [ ]:
def test_school_complexity2() -> None:
    from typing import Iterable
    import pandas as pd
    
    def club_member_count(members) -> Iterable[int]:
        yield len(members.split(','))
    
    def club_status(member_count, funding) -> Iterable[str]:
        if member_count >= 10 and funding >= 500:
            yield "Active"
        else:
            yield "Inactive"
    
    club_member_count_dict = dict(ie_function=club_member_count, ie_function_name='club_member_count', in_rel=[DataTypes.string], out_rel=[DataTypes.integer])
    club_status_dict = dict(ie_function=club_status, ie_function_name='club_status', in_rel=[DataTypes.integer, DataTypes.integer], out_rel=[DataTypes.string])
    
    commands = """
               new club(str, str, int)
               club("Science", "Alice,Bob,Carol,Dave", 600)
               club("Art", "Eve,Frank", 200)
               
               club_info(C, MC, STATUS) <- club(C, M, F), club_member_count(M) -> (MC), club_status(MC, F) -> (STATUS)
               
               ?club_info(C, MC, STATUS)
               """
    
    expected_result = {
        'C': ['Science', 'Art'],
        'MC': [4, 2],
        'STATUS': ['Inactive', 'Inactive']
    }
    
    expected_result_df = pd.DataFrame(expected_result)
    session = run_test_df_compare(commands, expected_result_df, [club_member_count_dict, club_status_dict])

test_school_complexity2()

In [ ]:
def test_school_complexity3() -> None:
    from typing import Iterable
    import pandas as pd
    
    def calculate_fine(days) -> Iterable[int]:
        yield days * 2
    
    def book_status(fine, condition) -> Iterable[str]:
        if fine > 0 and condition == "Damaged":
            yield "Pay Fine and Repair"
        elif fine > 0:
            yield "Pay Fine"
        elif condition == "Damaged":
            yield "Repair"
        else:
            yield "OK"
    
    calculate_fine_dict = dict(ie_function=calculate_fine, ie_function_name='calculate_fine', in_rel=[DataTypes.integer], out_rel=[DataTypes.integer])
    book_status_dict = dict(ie_function=book_status, ie_function_name='book_status', in_rel=[DataTypes.integer, DataTypes.string], out_rel=[DataTypes.string])
    
    commands = """
               new book(str, int, str)
               book("Math101", 5, "Good")
               book("Science101", 3, "Damaged")
               book("Art101", 0, "Good")
               
               book_info(B, F, STATUS) <- book(B, D, C), calculate_fine(D) -> (F), book_status(F, C) -> (STATUS)
               
               ?book_info(B, F, STATUS)
               """
    
    expected_result = {
        'B': ['Math101', 'Science101', 'Art101'],
        'F': [10,6,0],
        'STATUS': ["Pay Fine","Pay Fine and Repair", "OK"]
    }

    expected_result_df = pd.DataFrame(expected_result)

    session = run_test_df_compare(commands, expected_result_df, [calculate_fine_dict, book_status_dict])

test_school_complexity3()